Script for scraping old data from dhhs

In [2]:
# ensure that python can find my functions
import sys
sys.path.append('../scripts')

In [3]:
import pandas as pd
import numpy as np
from covid19_utils import get_active_cases
from covid19_utils import clean_active_cases
from datetime import date
from datetime import timedelta
import time

In [2]:
pd.options.mode.chained_assignment = None

collect data from June 14 to today. put into a single csv.

In [3]:
curr_date = date(2020, 6, 14)
end_date = date.today()

# get the df started with the first date, so all the index names are there
df_all_dates = get_active_cases(curr_date)
df_all_dates = clean_active_cases(df_all_dates, curr_date)

# now get the cases for the rest of the dates until yesterday
curr_date += timedelta(days=1)

while (curr_date < end_date):
    df_curr = get_active_cases(curr_date)
    df_curr = clean_active_cases(df_curr, curr_date)
    df_all_dates = df_all_dates.merge(df_curr, on = 'lga', how = 'outer')
    curr_date += timedelta(days=1)
    time.sleep(10)

In [6]:
# after the merge, lgas that weren't present in earlier tables have blanks. replace these with 0s.
df_all_dates.replace(np.nan, 0, inplace = True)

# order by index so the data for a specific LGA is easier to find
df_all_dates.sort_values(by='lga', axis=0, inplace=True)

In [7]:
df_all_dates.to_csv('../data/all_dates.csv')